# Exercise 4

In [64]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
import copy

import numpy as np
import unidecode
import string
import random
import time
import math

# Tensorboard for visualizing
from torch.utils.tensorboard import SummaryWriter

### Import Tiny Shakespeare

### Model from https://github.com/spro/char-rnn.pytorch/blob/master/model.py

In [65]:
class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, model="gru", n_layers=1):
        super(CharRNN, self).__init__()
        self.model = model.lower()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(input_size, hidden_size)
        if self.model == "gru":
            self.rnn = nn.GRU(hidden_size, hidden_size, n_layers)
        elif self.model == "lstm":
            self.rnn = nn.LSTM(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden):
        batch_size = input.size(0)
        encoded = self.encoder(input)
        output, hidden = self.rnn(encoded.view(1, batch_size, -1), hidden)
        output = self.decoder(output.view(batch_size, -1))
        return output, hidden

    def forward2(self, input, hidden):
        encoded = self.encoder(input.view(1, -1))
        output, hidden = self.rnn(encoded.view(1, 1, -1), hidden)
        output = self.decoder(output.view(1, -1))
        return output, hidden

    def init_hidden(self, batch_size):
        if self.model == "lstm":
            return (Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size)),
                    Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size)))
        return Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size))


### helpers.py from https://github.com/spro/char-rnn.pytorch/blob/master/helpers.py

In [66]:
# https://github.com/spro/char-rnn.pytorch

# Reading and un-unicode-encoding data

all_characters = string.printable
n_characters = len(all_characters)

def read_file(filename):
    file = unidecode.unidecode(open(filename).read())
    return file, len(file)

# Turning a string into a tensor

def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        try:
            tensor[c] = all_characters.index(string[c])
        except:
            continue
    return tensor

# Readable time elapsed

def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

### generate.py from https://github.com/spro/char-rnn.pytorch/blob/master/generate.py

In [79]:
#!/usr/bin/env python
# https://github.com/spro/char-rnn.pytorch

import torch
import os
import argparse

def generate(decoder, prime_str="A", predict_len=100, temperature=0.8, cuda=False):
    hidden = decoder.init_hidden(1)
    prime_words = make_trainset(prime_str, len(prime_str))
    start_seq = get_index_vocab(vocab,prime_words)
    prime_input = Variable(torch.as_tensor(start_seq).unsqueeze(0))

    if cuda:
        hidden = hidden.cuda()
        prime_input = prime_input.cuda()
    predicted = prime_str

    # Use priming string to "build up" hidden state
    for p in range(len(prime_words) - 1):
        _, hidden = decoder(prime_input[:,p], hidden)

    inp = prime_input[:,-1]

    for p in range(predict_len):
        output, hidden = decoder(inp, hidden)

        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]

        # Add predicted character to string and use as next input
        predicted_word = vocab[top_i]
        predicted += predicted_word + " "
        inp = Variable(torch.as_tensor(top_i).unsqueeze(0))
        if cuda:
            inp = inp.cuda()

    return predicted

# Run as standalone script
#if __name__ == '__main__':

# Parse command line arguments
#     argparser = argparse.ArgumentParser()
#     argparser.add_argument('filename', type=str)
#     argparser.add_argument('-p', '--prime_str', type=str, default='A')
#     argparser.add_argument('-l', '--predict_len', type=int, default=100)
#     argparser.add_argument('-t', '--temperature', type=float, default=0.8)
#     argparser.add_argument('--cuda', action='store_true')
#     args = argparser.parse_args()

filename = "Task4.pt"
start = "which is, "

decoder = torch.load(filename)
del filename
print(generate(decoder, prime_str=start))

which is, story 
 ruin. 
 
 
 
 
 
 wouldst and Which but 
 congealed 
 go 
 
 
 
 by As Lord quickly that not wrong. 
 Let needs or 
 for the so 
 
 is of a I A of the 
 
 
 
 the 
 the I III: what 
 a 
 the entreat I 
 
 
 
 A woman and 
 
 boy? 
 
 in have 
 
 
 we did 
 
 of yet so Would 
 no. or is my 
 in in come 
 III: 
 Cupid's brother's 


### train.py from https://github.com/spro/char-rnn.pytorch/blob/master/train.py

In [87]:
#!/usr/bin/env python
# https://github.com/spro/char-rnn.pytorch

import torch
import torch.nn as nn
from torch.autograd import Variable
import argparse
import os

from tqdm import tqdm


# Parse command line arguments
# argparser = argparse.ArgumentParser()
# argparser.add_argument('filename', type=str, default='tinyShakespear.txt')
# argparser.add_argument('--model', type=str, default="gru")
# argparser.add_argument('--n_epochs', type=int, default=2000)
# argparser.add_argument('--print_every', type=int, default=100)
# argparser.add_argument('--hidden_size', type=int, default=100)
# argparser.add_argument('--n_layers', type=int, default=2)
# argparser.add_argument('--learning_rate', type=float, default=0.01)
# argparser.add_argument('--chunk_len', type=int, default=200)
# argparser.add_argument('--batch_size', type=int, default=100)
# argparser.add_argument('--shuffle', action='store_true')
# argparser.add_argument('--cuda', action='store_true')
# args = argparser.parse_args()

arg_filename = "tinyShakespear.txt"
arg_model = "gru"
arg_n_epochs = 2000
arg_print_every = 2000
arg_hidden_size = 100
arg_n_layers = 2
arg_learning_rate = 0.01
arg_chunk_len = 20
arg_batch_size = 200
arg_shuffle = True
arg_cuda = False


if arg_cuda:
    print("Using CUDA")

file, file_len = read_file(arg_filename)

def make_trainset(input_seq, input_len):
    my_vocab = []
    my_word = ""
    for i in range(input_len):
        my_char = input_seq[i]
        if my_char == ' ' or my_char == '\0':
            if len(my_word) > 0:
                my_vocab.append(my_word)
            my_word = ""
        elif my_char == '\n':
            if len(my_word) > 0:
                my_vocab.append(my_word)
            my_vocab.append('\n')
            my_word = ""
        else:
            my_word = my_word + my_char
    if len(my_word) > 0:
        my_vocab.append(my_word)
    return my_vocab

def make_vocab(input_seq, input_len):
    my_vocab = ['\n']
    my_word = ""
    for i in range(input_len):
        my_char = input_seq[i]
        if my_char == ' ' or my_char == '\n' or my_char == '\0':
            if len(my_word) > 0:
                if my_word not in my_vocab:
                    my_vocab.append(my_word)
            my_word = ""
        else:
            my_word = my_word + my_char

    if len(my_word) > 0:
        if my_word not in my_vocab:
            my_vocab.append(my_word)

    return my_vocab

def get_index_vocab(my_vocab, chunk):
    index_list = []
    for word in chunk:
        try:
            index_list.append(my_vocab.index(word))
        except:
            continue

    return index_list

vocab = make_vocab(file, file_len)
trainset = make_trainset(file,file_len)

def random_training_set(chunk_len, batch_size):
    inp = torch.LongTensor(batch_size, chunk_len)
    target = torch.LongTensor(batch_size, chunk_len)
    for bi in range(batch_size):
        start_index = random.randint(0, len(trainset) - chunk_len)
        end_index = start_index + chunk_len + 1
        chunk = trainset[start_index:end_index]
        chunk = get_index_vocab(vocab, chunk)
        inp[bi] = torch.as_tensor(chunk[:-1])
        target[bi] = torch.as_tensor(chunk[1:])
    inp = Variable(inp)
    target = Variable(target)
    if arg_cuda:
        inp = inp.cuda()
        target = target.cuda()
    return inp, target

def train(inp, target):
    hidden = decoder.init_hidden(arg_batch_size)
    if arg_cuda:
        hidden = hidden.cuda()
    decoder.zero_grad()
    loss = 0

    for c in range(arg_chunk_len):
        output, hidden = decoder(inp[:,c], hidden)
        loss += criterion(output.view(arg_batch_size, -1), target[:,c])

    loss.backward()
    decoder_optimizer.step()

    return loss.item()/ arg_chunk_len

def save():
    save_filename = 'Task4.pt'
    torch.save(decoder, save_filename)
    print('Saved as %s' % save_filename)

# Initialize models and start training

decoder = CharRNN(
    len(vocab),
    arg_hidden_size,
    len(vocab),
    model = arg_model,
    n_layers = arg_n_layers
)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=arg_learning_rate)
criterion = nn.CrossEntropyLoss()

if arg_cuda:
    decoder.cuda()

start = time.time()
all_losses = []
loss_avg = 0

writer = SummaryWriter()

try:
    print("Training for %d epochs..." % arg_n_epochs)
    for epoch in tqdm(range(1, arg_n_epochs + 1)):
        loss = train(*random_training_set(arg_chunk_len, arg_batch_size))
        writer.add_scalar("Perplexity", np.exp(loss), epoch)
        loss_avg += loss

        if epoch % arg_print_every == 0:
            print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / arg_n_epochs * 100, loss))
            print(generate(decoder, 'Wh', 100, cuda=arg_cuda), '\n')

    print("Saving...")
    save()

except KeyboardInterrupt:
    print("Saving before quit...")
    save()

writer.flush()

Training for 2000 epochs...


100%|█████████▉| 1999/2000 [1:23:00<00:02,  2.49s/it]

 0s (2000 100%) 2.9205]


IndexError: index -1 is out of bounds for dimension 1 with size 0

![Picture missing](img_1.png "Validation accuracy each epoch")

In [82]:
filename = "Task4.pt"
start = "The "

decoder = torch.load(filename)
del filename
print(generate(decoder, prime_str=start))

The vineyard by the to have you 
 it 
 rehearse, not 
 
 A the 
 
 none 
 is you 
 to 
 of then some what even obstacles: three peace: 
 with 
 
 
 to if 
 
 
 to 
 
 
 time 
 is 
 that the he of 
 their to God I 
 
 
 be York 
 give 
 
 of his 
 THOMAS king. of 
 
 
 restored your 
 
 
 LEONTES: 
 thy since 
 
 against 
 
 up 
 benefit 
 of 
 my on 
 


In [83]:
start = ","

print(generate(decoder, prime_str=start))

IndexError: index -1 is out of bounds for dimension 1 with size 0

In [84]:
start = "which is,"

print(generate(decoder, prime_str=start))

which is,Of Signior to man off 
 
 
 
 this GLOUCESTER: Forgive the 
 the know the he this 
 lord; So 
 set 
 
 
 to good 
 man's wants, make not and 
 handsome to draw 
 and for 
 
 O, 
 
 
 LUCIO: But 
 mind 
 
 
 the 
 to 
 till 
 
 is dear imagine 
 so they 
 
 
 he 
 
 this the And have 
 the friendly Iniquity? 
 
 
 
 me 
 
 their 
 will 
 fault 
 
 it shall thick-ribbed 
 


In [85]:
start = "blah blah blah"

print(generate(decoder, prime_str=start))


IndexError: index 0 is out of bounds for dimension 1 with size 0